In [5]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# 读取数据
file_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist指数计算数据.xlsx'
df = pd.read_excel(file_path)

# 输入和输出列
input_columns = ['土地', '水资源', '固废', '二氧化硫', '烟（粉）尘', '废水', '化学需氧量']
output_columns = ['GDP']

# 提取年份和城市
years = df['年份'].unique()
cities = df['城市'].unique()

# 初始化存储结果
efficiency_scores = []

# DEA 计算函数（BCC 输入导向）
def dea_bcc_input(input_data, output_data):
    num_dmu = len(input_data)  # 决策单元数
    num_inputs = input_data.shape[1]  # 投入变量数
    num_outputs = output_data.shape[1]  # 产出变量数
    
    efficiency = []
    
    for i in range(num_dmu):
        # 当前 DMU 的投入和产出
        target_input = input_data[i]
        target_output = output_data[i]
        
        # 构造目标函数
        c = np.hstack([np.zeros(num_dmu), -1])  # lambda 和 theta
        
        # 构造约束矩阵
        A_ub = np.zeros((num_inputs + num_outputs, num_dmu + 1))
        b_ub = np.zeros(num_inputs + num_outputs)
        
        # 输入约束：-lambda * X + theta * x_0 <= 0
        for j in range(num_dmu):
            A_ub[:num_inputs, j] = -input_data[j]
        A_ub[:num_inputs, -1] = target_input
        
        # 输出约束：lambda * Y >= y_0
        for j in range(num_dmu):
            A_ub[num_inputs:num_inputs + num_outputs, j] = output_data[j]
        A_ub[num_inputs:num_inputs + num_outputs, -1] = -target_output
        
        # 线性规划求解
        bounds = [(0, None)] * num_dmu + [(None, None)]  # lambda >= 0, theta free
        result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
        
        if result.success:
            efficiency.append(1 / result.x[-1])  # DEA 效率得分
        else:
            efficiency.append(None)  # 求解失败返回 None
    
    return efficiency

# 按年份计算效率得分
for year in years:
    data_year = df[df['年份'] == year]
    inputs = data_year[input_columns].values
    outputs = data_year[output_columns].values
    
    # 计算效率得分
    scores = dea_bcc_input(inputs, outputs)
    
    # 保存结果
    for city, score in zip(data_year['城市'], scores):
        efficiency_scores.append({'城市': city, '年份': year, '效率得分': score})

# 转换为 DataFrame
efficiency_df = pd.DataFrame(efficiency_scores)

# 保存效率得分结果
efficiency_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/efficiency_scores.xlsx'
efficiency_df.to_excel(efficiency_path, index=False)
print(f"效率得分已保存至：{efficiency_path}")

效率得分已保存至：/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/efficiency_scores.xlsx


In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# 读取数据
file_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist指数计算数据.xlsx'
df = pd.read_excel(file_path)

# 输入和输出列
input_columns = ['土地', '水资源', '固废', '二氧化硫', '烟（粉）尘', '废水', '化学需氧量']
output_columns = ['GDP']

inputs = data_year[input_columns].values
outputs = data_year[output_columns].values

# 归一化数据
inputs = inputs / inputs.max(axis=0)
outputs = outputs / outputs.max(axis=0)

# 提取年份和城市
years = df['年份'].unique()
cities = df['城市'].unique()

# 初始化存储结果
efficiency_scores = []

# DEA 计算函数（BCC 输入导向）
def dea_bcc_input(input_data, output_data):
    num_dmu = len(input_data)  # 决策单元数
    num_inputs = input_data.shape[1]  # 投入变量数
    num_outputs = output_data.shape[1]  # 产出变量数
    
    efficiency = []
    
    for i in range(num_dmu):
        # 当前 DMU 的投入和产出
        target_input = input_data[i]
        target_output = output_data[i]
        
        # 构造目标函数
        c = np.hstack([np.zeros(num_dmu), -1])  # lambda 和 theta
        
        # 构造约束矩阵
        A_ub = np.zeros((num_inputs + num_outputs, num_dmu + 1))
        b_ub = np.zeros(num_inputs + num_outputs)
        
        # 输入约束：-lambda * X + theta * x_0 <= 0
        for j in range(num_dmu):
            A_ub[:num_inputs, j] = -input_data[j]
        A_ub[:num_inputs, -1] = target_input
        
        # 输出约束：lambda * Y >= y_0
        for j in range(num_dmu):
            A_ub[num_inputs:num_inputs + num_outputs, j] = output_data[j]
        A_ub[num_inputs:num_inputs + num_outputs, -1] = -target_output
        
        # 线性规划求解
        bounds = [(0, None)] * num_dmu + [(None, None)]  # lambda >= 0, theta free
        result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
        
        if result.success:
            efficiency.append(1 / result.x[-1])  # DEA 效率得分
        else:
            print(f"DMU {i} 求解失败")
            efficiency.append(None)  # 求解失败返回 None
    
    return efficiency

# 按年份计算效率得分
for year in years:
    data_year = df[df['年份'] == year]
    inputs = data_year[input_columns].values
    outputs = data_year[output_columns].values

    # 打印检查数据
    print(f"年份 {year} 的输入数据：\n", inputs)
    print(f"年份 {year} 的输出数据：\n", outputs)

    # 数据归一化（推荐）
    inputs = inputs / inputs.max(axis=0)
    outputs = outputs / outputs.max(axis=0)

    # 计算效率得分
    scores = dea_bcc_input(inputs, outputs)
    
    # 保存结果
    for city, score in zip(data_year['城市'], scores):
        efficiency_scores.append({'城市': city, '年份': year, '效率得分': score})

# 转换为 DataFrame
efficiency_df = pd.DataFrame(efficiency_scores)

# 保存效率得分结果
efficiency_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/efficiency_scores.xlsx'
efficiency_df.to_excel(efficiency_path, index=False)
print(f"效率得分已保存至：{efficiency_path}")

年份 2006 的输入数据：
 [[3.37715107e+01 3.45000000e+01 1.35600000e+03 9.40000000e+00
  4.50000000e+00 1.01700000e+04 9.25770000e-01]
 [3.68188813e+01 2.30900000e+01 1.29200000e+03 2.32000000e+01
  7.70000000e+00 2.29780000e+04 3.68747000e+00]
 [1.79420187e+02 2.01780000e+02 1.42290000e+04 1.32600000e+02
  1.19900000e+02 1.30340000e+05 3.58180900e+01]
 [8.69449440e+01 5.57200000e+01 1.18170000e+04 1.17700000e+02
  1.48800000e+02 4.40910000e+04 1.69633800e+01]
 [1.49244393e+02 1.74760000e+02 8.71000000e+03 1.38400000e+02
  7.58000000e+01 2.78230000e+04 1.36113800e+01]
 [1.39882153e+02 1.33260000e+02 1.30130000e+04 1.03700000e+02
  8.76000000e+01 9.47240000e+04 2.60792000e+01]
 [1.06530089e+02 9.83900000e+01 2.80200000e+03 3.36000000e+01
  4.56000000e+01 3.93210000e+04 1.67988900e+01]
 [1.49244727e+02 2.71510000e+02 3.91400000e+03 4.40000000e+01
  5.68000000e+01 4.48010000e+04 1.41735600e+01]
 [2.53513027e+01 1.21280000e+02 2.06300000e+03 3.74000000e+01
  5.70000000e+00 4.83360000e+04 3.52762000

In [9]:
# 整理效率得分表
efficiency_pivot = efficiency_df.pivot(index='城市', columns='年份', values='效率得分').fillna(1)

# 存储 Malmquist 指数结果
malmquist_results = []

# 遍历年份，计算 EC 和 TC
for i in range(1, len(years)):
    prev_year = years[i-1]
    current_year = years[i]

    # 提取上一年和当前年的效率得分
    prev_eff = efficiency_pivot[prev_year]
    current_eff = efficiency_pivot[current_year]

    # 计算效率变化（EC）
    EC = current_eff / prev_eff

    # 技术变化（TC）：这里假设 TC = 1，具体可基于跨期效率分析补充计算
    TC = np.ones_like(EC)

    # 计算 Malmquist 指数
    malmquist_index = EC * TC

    # 保存结果
    for city, ec, tc, mi in zip(cities, EC, TC, malmquist_index):
        malmquist_results.append({
            '城市': city,
            '年份': f"{prev_year}-{current_year}",
            'EC': ec,
            'TC': tc,
            'Malmquist': mi
        })

# 转换为 DataFrame
malmquist_df = pd.DataFrame(malmquist_results)

# 输出到 Excel
malmquist_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist_index_results.xlsx'
malmquist_df.to_excel(malmquist_path, index=False)
print(f"Malmquist 指数已保存至：{malmquist_path}")

Malmquist 指数已保存至：/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist_index_results.xlsx


/var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/ipykernel_96069/2999785693.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  efficiency_pivot = efficiency_df.pivot(index='城市', columns='年份', values='效率得分').fillna(1)


In [10]:
import pandas as pd
import numpy as np

# 读取Excel数据
file_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist指数计算数据（效率得分）.xlsx'
df = pd.read_excel(file_path)

# 按年份和城市分组，确保数据正确排序
df = df.sort_values(by=['年份', '城市'])

# 准备结果存储
results = []

# 获取年份和城市列表
years = df['年份'].unique()
cities = df['城市'].unique()

# 投入和产出列
input_columns = ['土地', '水资源', '固废', '二氧化硫', '烟（粉）尘', '废水', '化学需氧量']
output_columns = ['GDP']

# 初始化效率矩阵（模拟数据或从DEA模型导入实际计算得分）
# 这里假设 df 中含有效率得分（如'综合效益'列）
efficiency_scores = df.pivot(index='城市', columns='年份', values='效率得分').values

# 遍历年份，计算 EC, TC 和 Malmquist 指数
for t in range(1, len(years)):
    prev_year = years[t-1]
    current_year = years[t]

    # 当前年份和上一年的效率得分
    prev_efficiency = efficiency_scores[:, t-1]
    current_efficiency = efficiency_scores[:, t]

    # 计算效率变化（EC）
    EC = current_efficiency / prev_efficiency

    # 技术变化（TC）：暂用1（需要额外计算）
    # 在实际中，需要基于当前和上一年前沿计算D_t和D_t-1。
    TC = np.ones_like(EC)

    # 计算 Malmquist 指数
    Malmquist_index = EC * TC

    # 保存结果
    for i, city in enumerate(cities):
        results.append({
            '城市': city,
            '年份': f"{prev_year}-{current_year}",
            'EC': EC[i],
            'TC': TC[i],
            'Malmquist': Malmquist_index[i]
        })

# 转为 DataFrame
results_df = pd.DataFrame(results)

# 输出到 Excel
output_path = '/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist_index_results.xlsx'
results_df.to_excel(output_path, index=False)

print(f"计算结果已保存至：{output_path}")

计算结果已保存至：/Users/niuko/作业/供应链成本与绩效管理/课程论文/DEA数据/malmquist_index_results.xlsx
